In [1]:
import pandas as pd
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

def spotify_login(cid, secret):
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager)

cid = "2758fb4fd4694c6ba9b71110bde0f9b1"
secret = "9ea1abff9d7f4fe7877b619eb890df9c"

sp = spotify_login(cid, secret)

In [4]:
# Read in the dataframe
path = 'data/lists of songs/'
songs = pd.read_csv(path + 'dataframe_songnames_by_country_and_year.csv') 

df = songs[['songs', 'artist', 'title']]

In [5]:

# Define the song features to extract
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']

# Create a list to store the rows to drop
rows_to_drop = []

# Loop through the rows in the dataframe
for index, row in df.iterrows():
    # Search for the track using the song name and artist
    results = sp.search(q='track:' + row['songs'] + ' artist:' + row['artist'], type='track')
    items = results['tracks']['items']
    if len(items) > 0:
        track_id = items[0]['id']
        # Get the audio features for the track
        audio_features = sp.audio_features(track_id)[0]
        # Append the song features and popularity to the dataframe
        for feature in features:
            df.loc[index, feature] = audio_features[feature]
        df.loc[index, 'popularity'] = items[0]['popularity']
    else:
        rows_to_drop.append(index)

In [8]:
# Drop rows with empty track_id or missing features
df.dropna(subset=features, inplace=True)

# Save dataframe in a csv file
df.to_csv("data/lists of songs/dataframe_song_features.csv")